In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
import seaborn as sns



## alle parts en items over de gehele tijd en alle linacs

In [39]:
linac2_headers = pd.read_csv("data/calblocks_update_table_cal_headers.csv", names=["date", "file_name", "empty", "thiing", "thing"])
linac2_items = pd.read_csv("data/calblocks_update_table_cal_items.csv",  names=["item", "category", "thing", "thiing"])
linac2_values = pd.read_csv("data/calblocks_update_table_cal_values.csv", names=["id", "calheaders_id", "item", "part", "value"])

linac2_headers.reset_index(inplace=True)
linac2_headers.set_index("level_0", inplace=True)
linac2_headers['datum'] = linac2_headers['date'].str.replace('/', '-', regex=False)

linac2_headers['date'] = pd.to_datetime(linac2_headers['date'], format='%d/%m/%Y')
linac2_headers["date"] = linac2_headers["date"].astype("str")

linac2_headers.columns = ["linac","date", "file_name", "empty", "thiing", "thing", "datum"]

## Item 381, gun aim current

In [40]:
tempdf = linac2_headers[["linac", "date", "file_name"]].merge(linac2_values[["calheaders_id", "item", "part", "value"]] ,left_on="level_0", right_on="calheaders_id", how="inner").drop(columns="calheaders_id")
merged_df = tempdf.merge(linac2_items[["item", "category"]], on = "item", how="inner")


be5201 = merged_df[merged_df["file_name"] == "Be5201"]

be5201 = be5201.sort_values(by=['date', 'linac'])
be5201["Part&item"] = "I:"+ be5201["item"].astype(str) + " P:"+ be5201["part"].astype(str)

px.line(be5201[be5201["item"] == 381], x="date", y ="value", color="linac", hover_data="date", markers=True)

In [42]:
df2191 = be5201[be5201["linac"] == 2191]
df2191["date"] = df2191["date"].astype("str")

px.line(df2191, x="date", color = "category", y ="value", hover_data="date", markers=True)

C:\Users\Laris\AppData\Local\Temp\ipykernel_17976\1741293812.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [167]:
dates = list(set(be5201["date"]))

for item in range(len(dates)-1):
    old = be5201[be5201["date"] == dates[item]]
    new = be5201[be5201["date"] == dates[item+1]]

    new["value"].values - old["value"].values

    
    

ValueError: operands could not be broadcast together with shapes (228,) (456,) 

De

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest


In [80]:

df_ml = no_p45[["value", "Part&item", 'date', "linac"]].pivot(index=['date', "linac"], columns='Part&item', values='value') 

y = df_ml.iloc[:,:-5]["I:381 P:1"]
X = df_ml.iloc[:,:-5].drop("I:381 P:1", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
	

In [81]:
models= [
    LinearRegression,
    RandomForestRegressor,
    SVR,
    GaussianProcessRegressor,
    BayesianRidge
]

metric_scores = {}
for model in models:
    scores = cross_validate(model(), X, y, return_train_score=True)
    for key, val in scores.items():
        scores[key] = val.mean()
    metric_scores[f"{model.__name__}"] = scores
    
pd.DataFrame(metric_scores).T

,fit_time,score_time,test_score,train_score
LinearRegression,0.004824,0.001348,-317.928225,0.999657
RandomForestRegressor,0.084012,0.008086,0.739990,0.996662
SVR,0.003488,0.001369,-2.918310,0.256159
GaussianProcessRegressor,0.002995,0.001740,-3974.311787,1.000000
BayesianRidge,0.003324,0.001309,-4080.235010,0.999313


In [86]:
metric_scores = {}
k = 76
while k:
    X_select = SelectKBest(k=k).fit_transform(X, y)
    scores = cross_validate(RandomForestRegressor(), X_select, y, return_train_score=True)
    for key, val in scores.items():
        scores[key] = val.mean()
    metric_scores[f"{k} features"] = scores
    k -= 1

kbest = pd.DataFrame(metric_scores).T


c:\Users\Laris\miniconda3\envs\stage\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning:

Features [ 1  7  8 12 13 14 15 16 17 18 19 20 21 22 23 24 28 29 30 31 32 33 34 35
 36 37 38 39 40 49 50 51 52 53 54 55 56 57 58 59 60 61 63 64 65 66 67 68
 69 70 71 72 73] are constant.

c:\Users\Laris\miniconda3\envs\stage\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning:

divide by zero encountered in divide

c:\Users\Laris\miniconda3\envs\stage\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning:

invalid value encountered in divide

c:\Users\Laris\miniconda3\envs\stage\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning:

Features [ 1  7  8 12 13 14 15 16 17 18 19 20 21 22 23 24 28 29 30 31 32 33 34 35
 36 37 38 39 40 49 50 51 52 53 54 55 56 57 58 59 60 61 63 64 65 66 67 68
 69 70 71 72 73] are constant.

c:\Users\Laris\miniconda3\envs\stage\Lib\site

In [ ]:
kbest[kbest["test_score"] > 0.739990].sort_values(by="test_score", ascending=False)

,fit_time,score_time,test_score,train_score
2 features,0.209074,0.019277,0.820197,0.998134
4 features,0.209232,0.018031,0.799703,0.997672
3 features,0.223980,0.020797,0.795667,0.997917
5 features,0.200119,0.017848,0.790065,0.997256
6 features,0.236229,0.019025,0.767550,0.997230
72 features,0.229624,0.019569,0.752759,0.997245
66 features,0.239345,0.020172,0.752632,0.997001
58 features,0.220017,0.019096,0.751621,0.996477
64 features,0.227282,0.026622,0.750137,0.996406
63 features,0.231796,0.020398,0.748568,0.996681


In [ ]:
model = RandomForestRegressor().fit(X, y)

model.predict()